In [8]:
import os
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import csv

class LabelingApp:
    def __init__(self, root, folder_path, csv_file_output):
        self.root = root
        self.folder_path = folder_path
        self.csv_file_output = csv_file_output
        self.images = self.load_images(folder_path)
        self.current_index = 0
        self.labels = [{"image": img, "values": [None]*8} for img in self.images]
        self.photos = [] 

        self.setup_ui()
        self.load_content()

        # Bind the window close event to on_closing method
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)


    def load_images(self, folder_path):
        return [f for f in sorted(os.listdir(folder_path)) if f.endswith('.jpg')]

    def setup_ui(self):
        self.root.title("Labeling App")

        # Configure grid layout behavior
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_columnconfigure(1, weight=3)

        # Progress display at the top
        self.progress_label = tk.Label(self.root, text="Progress: 0/0")
        self.progress_label.grid(row=0, column=0, columnspan=2, sticky='ew')

        # Image display
        self.image_label = tk.Label(self.root)
        self.image_label.grid(row=1, column=0, sticky='nsew')

        # Creating a frame for entries on the right side
        entry_frame = tk.Frame(self.root)
        entry_frame.grid(row=1, column=1, sticky='n')

        # Entry widgets for the labels
        self.entries = []
        for i in range(8):
            entry_label = tk.Label(entry_frame, text=f"{(i)*45}-{(i+1)*45} degree:")
            entry_label.grid(row=i, column=0, sticky='e')
            entry = tk.Entry(entry_frame, width=10)
            entry.grid(row=i, column=1, sticky='w')
            self.entries.append(entry)

        # Navigation buttons at the bottom
        nav_frame = tk.Frame(self.root)
        nav_frame.grid(row=2, column=0, columnspan=2, sticky='ew')
        self.prev_button = ttk.Button(nav_frame, text="Previous", command=self.prev_content)
        self.prev_button.pack(side=tk.LEFT)
        self.next_button = ttk.Button(nav_frame, text="Next", command=self.next_content)
        self.next_button.pack(side=tk.RIGHT)


    def load_content(self):
        if self.images:
            image_path = os.path.join(self.folder_path, self.images[self.current_index])
            image = Image.open(image_path)

            # Resize the image to half of its original dimensions
            original_size = image.size
            resized_size = (original_size[0] // 2, original_size[1] // 2)
            image = image.resize(resized_size, Image.LANCZOS)

            # Create a PhotoImage and store it to prevent garbage collection
            photo = ImageTk.PhotoImage(image)
            self.photos.append(photo)  # Save the reference
            self.image_label.config(image=photo)
            self.image_label.image = photo  # Save reference

            # Update progress label
            progress_text = f"Progress: {self.current_index + 1}/{len(self.images)}"
            self.progress_label.config(text=progress_text)

            # Reset entries
            for entry in self.entries:
                entry.delete(0, tk.END)
                entry.insert(0, '')
        

    def save_labels(self, index):
        self.labels[index]["values"] = [entry.get() for entry in self.entries]

    def prev_content(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.load_content()

    def next_content(self):
        if self.current_index < len(self.images) - 1:
            self.save_labels(self.current_index)
            self.current_index += 1
            self.load_content()

    def on_closing(self):
        self.save_labels(self.current_index)
        self.save_to_file()
        self.root.destroy()

    def save_to_file(self):
        with open(self.csv_file_output, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['id', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8'])
            for label in self.labels:
                image_number = os.path.splitext(label['image'])[0]
                writer.writerow([image_number] + label['values'])


ModuleNotFoundError: No module named 'PIL'

In [ ]:

if __name__ == "__main__":
    root = tk.Tk()
    folder_path = "backend/services/result6/holds" 
    csv_file_output = "backend/services/result6/difficulties.csv" 
    app = LabelingApp(root, folder_path, csv_file_output)
    root.mainloop()
